
### Remove records with NULL customer_id

In [0]:
%sql
select * from gizmobox.bronze.v_customers where customer_id is not null;


### remove exact duplicate records

In [0]:
%sql

create or replace temporary view v_customers_distinct 
as 
select distinct * from gizmobox.bronze.v_customers where customer_id is not null;

In [0]:
%sql
select customer_id, max(created_timestamp), max(customer_name), 
max(date_of_birth), max(email), max(member_since), max(telephone), max(file_path) 
from gizmobox.bronze.v_customers
where customer_id is not null group by customer_id;

-- use cte for better results

In [0]:
%sql
select customer_id, max(created_timestamp) 
from v_customers_distinct 
group by customer_id


### cast the columns values to the correct data type

In [0]:
%sql 
with cte_max as (
    select customer_id, max(created_timestamp) as max_timestamp
    from v_customers_distinct group by customer_id
)

select cast(t.created_timestamp as timestamp) as created_timestamp,
t.customer_id, t.customer_name, cast(t.date_of_birth as date) as date_of_birth, t.email, 
cast(t.member_since as date) as member_since, t.telephone
from v_customers_distinct t
inner join cte_max m on t.customer_id = m.customer_id and t.created_timestamp = m.max_timestamp;


### Write data to a delta table

In [0]:
%sql
create table gizmobox.silver.customers as 
with cte_max as (
    select customer_id, max(created_timestamp) as max_timestamp
    from v_customers_distinct group by customer_id
)

select cast(t.created_timestamp as timestamp) as created_timestamp,
t.customer_id, t.customer_name, cast(t.date_of_birth as date) as date_of_birth, t.email, 
cast(t.member_since as date) as member_since, t.telephone
from v_customers_distinct t
inner join cte_max m on t.customer_id = m.customer_id and t.created_timestamp = m.max_timestamp;

In [0]:
%sql
select * from gizmobox.silver.customers;

In [0]:
%sql describe extended gizmobox.silver.customers;